In [1]:
# STL Processing - Emulating Blender Actions

#Convert pre-processed GeoTIFF heightmap to 3D mesh for STL export

In [2]:
import numpy as np
import rasterio
from scipy.interpolate import RectBivariateSpline
import struct
import json

def import_raster_as_heightmap(input_tiff, scale_z=1.0):
    """
    Import a raster file as a heightmap (elevation data).
    
    Parameters:
    -----------
    input_tiff : str
        Path to the input pre-processed TIFF file
    scale_z : float
        Vertical exaggeration factor for elevation (default: 1.0)
    
    Returns:
    --------
    dict : Dictionary with heightmap data and metadata
        - 'heightmap': numpy array of elevation values
        - 'width': raster width in pixels
        - 'height': raster height in pixels
        - 'bounds': geographic bounds (minx, miny, maxx, maxy)
        - 'resolution': (pixel_size_x, pixel_size_y)
        - 'crs': coordinate reference system
        - 'min_elevation': minimum elevation value
        - 'max_elevation': maximum elevation value
    """
    
    with rasterio.open(input_tiff) as src:
        # Read elevation data (first band)
        heightmap = src.read(1)
        
        # Apply vertical exaggeration
        heightmap = heightmap * scale_z
        
        # Get metadata
        metadata = {
            'heightmap': heightmap,
            'width': src.width,
            'height': src.height,
            'bounds': src.bounds,
            'resolution': src.res,
            'crs': str(src.crs),
            'min_elevation': float(np.nanmin(heightmap)),
            'max_elevation': float(np.nanmax(heightmap)),
            'transform': src.transform
        }
    
    return metadata


def heightmap_to_mesh(heightmap_data, resolution=1.0, smooth=False):
    """
    Convert a heightmap to a 3D mesh in Blender-style format.
    Emulates Blender's "Landscape" add-on behavior.
    
    Parameters:
    -----------
    heightmap_data : dict
        Dictionary returned from import_raster_as_heightmap()
    resolution : float
        Mesh resolution (1.0 = use all pixels, 2.0 = skip every other pixel, etc.)
    smooth : bool
        Apply smoothing to the mesh (default: False)
    
    Returns:
    --------
    dict : Dictionary with mesh data
        - 'vertices': list of [x, y, z] vertex coordinates
        - 'faces': list of [v1, v2, v3] face triangles
        - 'vertex_count': total number of vertices
        - 'face_count': total number of faces
        - 'bounds': mesh bounding box
    """
    
    heightmap = heightmap_data['heightmap']
    pixel_width, pixel_height = heightmap_data['resolution']
    
    # Apply resolution decimation if needed
    step = int(resolution)
    if step > 1:
        heightmap = heightmap[::step, ::step]
    
    height, width = heightmap.shape
    
    # Generate vertex grid
    vertices = []
    
    # Create mesh vertices from heightmap
    for y in range(height):
        for x in range(width):
            # Geographic coordinates
            world_x = x * pixel_width * resolution
            world_y = y * pixel_height * resolution
            world_z = heightmap[y, x]
            
            vertices.append([world_x, world_y, world_z])
    
    # Generate faces (triangles from quad grid)
    faces = []
    
    for y in range(height - 1):
        for x in range(width - 1):
            # Current quad indices
            v0 = y * width + x
            v1 = y * width + (x + 1)
            v2 = (y + 1) * width + x
            v3 = (y + 1) * width + (x + 1)
            
            # Split quad into two triangles
            faces.append([v0, v1, v2])
            faces.append([v1, v3, v2])
    
    # Calculate bounding box
    vertices_array = np.array(vertices)
    bounds = {
        'min_x': float(np.min(vertices_array[:, 0])),
        'max_x': float(np.max(vertices_array[:, 0])),
        'min_y': float(np.min(vertices_array[:, 1])),
        'max_y': float(np.max(vertices_array[:, 1])),
        'min_z': float(np.min(vertices_array[:, 2])),
        'max_z': float(np.max(vertices_array[:, 2]))
    }
    
    mesh_data = {
        'vertices': vertices,
        'faces': faces,
        'vertex_count': len(vertices),
        'face_count': len(faces),
        'bounds': bounds
    }
    
    return mesh_data


# Example usage:
#heightmap = import_raster_as_heightmap('data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.tif', scale_z=1.0)
#print(f"Heightmap loaded: {heightmap['width']}x{heightmap['height']}")
# 
#mesh = heightmap_to_mesh(heightmap, resolution=1.0, smooth=False)
#print(f"Mesh created: {mesh['vertex_count']} vertices, {mesh['face_count']} faces")

## Convert Heightmap to 3D Mesh

Creates triangulated mesh from heightmap elevation data:
- **Vertices**: Grid of 3D points (x, y, z) from heightmap
- **Faces**: Triangulated surface connecting vertices
- **Mesh Resolution**: Option to decimate for performance
- **Output**: Ready for STL export or Blender import

In [3]:
import struct
import os

def export_mesh_to_stl(mesh_data, output_stl, ascii_format=False):
    """
    Export 3D mesh to STL format for 3D printing or CAD applications.
    
    Parameters:
    -----------
    mesh_data : dict
        Dictionary returned from heightmap_to_mesh()
    output_stl : str
        Path to output STL file
    ascii_format : bool
        If True, export as ASCII STL (larger file)
        If False, export as binary STL (smaller, faster) - default
    
    Returns:
    --------
    dict : Export info (file path, file size, vertex/face count)
    """
    
    vertices = mesh_data['vertices']
    faces = mesh_data['faces']
    
    if ascii_format:
        # ASCII STL format
        with open(output_stl, 'w') as f:
            f.write("solid geometry\n")
            
            for face in faces:
                v0, v1, v2 = face
                p0 = np.array(vertices[v0])
                p1 = np.array(vertices[v1])
                p2 = np.array(vertices[v2])
                
                # Calculate normal vector
                edge1 = p1 - p0
                edge2 = p2 - p0
                normal = np.cross(edge1, edge2)
                norm = np.linalg.norm(normal)
                if norm > 0:
                    normal = normal / norm
                
                f.write(f"  facet normal {normal[0]:.6e} {normal[1]:.6e} {normal[2]:.6e}\n")
                f.write("    outer loop\n")
                f.write(f"      vertex {p0[0]:.6e} {p0[1]:.6e} {p0[2]:.6e}\n")
                f.write(f"      vertex {p1[0]:.6e} {p1[1]:.6e} {p1[2]:.6e}\n")
                f.write(f"      vertex {p2[0]:.6e} {p2[1]:.6e} {p2[2]:.6e}\n")
                f.write("    endloop\n")
                f.write("  endfacet\n")
            
            f.write("endsolid geometry\n")
    else:
        # Binary STL format (more efficient)
        with open(output_stl, 'wb') as f:
            # 80-byte header
            header = b'Binary STL file - Generated from heightmap' + b'\0' * (80 - 42)
            f.write(header)
            
            # Number of triangles
            f.write(struct.pack('<I', len(faces)))
            
            for face in faces:
                v0, v1, v2 = face
                p0 = np.array(vertices[v0], dtype=np.float32)
                p1 = np.array(vertices[v1], dtype=np.float32)
                p2 = np.array(vertices[v2], dtype=np.float32)
                
                # Calculate normal
                edge1 = p1 - p0
                edge2 = p2 - p0
                normal = np.cross(edge1, edge2)
                norm = np.linalg.norm(normal)
                if norm > 0:
                    normal = normal / norm
                else:
                    normal = np.array([0, 0, 1], dtype=np.float32)
                
                # Write normal
                f.write(struct.pack('<fff', *normal.astype(np.float32)))
                
                # Write vertices
                f.write(struct.pack('<fff', *p0))
                f.write(struct.pack('<fff', *p1))
                f.write(struct.pack('<fff', *p2))
                
                # Attribute byte count
                f.write(struct.pack('<H', 0))
    
    # Get file size
    file_size = os.path.getsize(output_stl)
    
    return {
        'output_path': output_stl,
        'file_size_mb': round(file_size / (1024 * 1024), 3),
        'format': 'ASCII' if ascii_format else 'Binary',
        'vertex_count': mesh_data['vertex_count'],
        'face_count': mesh_data['face_count']
    }

# Example workflow:
# 1. Import raster as heightmap
#heightmap = import_raster_as_heightmap('data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.tif', scale_z=1.0)
#print(f"Loaded: {heightmap['width']}x{heightmap['height']} - Elevation range: {heightmap['min_elevation']:.1f}m to {heightmap['max_elevation']:.1f}m")
#
# 2. Convert to mesh
#mesh = heightmap_to_mesh(heightmap, resolution=1.0)
# print(f"Mesh created: {mesh['vertex_count']} vertices, {mesh['face_count']} faces")
#
# 3. Export to STL
#stl_info = export_mesh_to_stl(mesh, 'output_model.stl', ascii_format=False)
#print(f"STL exported: {stl_info['output_path']}")



## Better Approach: Displacement-Based Solid Model for 3D Printing

Instead of direct mesh conversion, use displacement mapping with a subdivided grid for better control and solid geometry creation.

In [4]:
from scipy.interpolate import RectBivariateSpline
from scipy.ndimage import binary_erosion, distance_transform_edt, gaussian_filter

def create_displaced_grid_model(heightmap_data, grid_subdivisions=1000, relief_height_mm=20.0, 
                                base_thickness=5.0, model_size=100.0, nodata_threshold=0.1, 
                                edge_blend_distance=10):
    """
    Create a 3D printable solid model using displacement mapping.
    This approach provides better control over geometry and creates a solid model suitable for FDM printing.
    
    Parameters:
    -----------
    heightmap_data : dict
        Dictionary returned from import_raster_as_heightmap()
        Contains 'heightmap', 'width', 'height', and other metadata
    grid_subdivisions : int
        Number of subdivisions in the grid (default: 1000x1000)
        Higher values = more detail but larger file
    relief_height_mm : float
        Desired height difference between lowest and highest point in mm (default: 20.0)
        This makes vertical exaggeration intuitive - larger = more dramatic terrain
        Set to None to use vertical_scale instead
    base_thickness : float
        Thickness of the solid base in mm (default: 5.0)
    model_size : float
        Size of the model in XY plane in mm (default: 100.0mm = 10cm square)
    nodata_threshold : float
        Heightmap values below this are treated as NoData (default: 0.1 meters)
        Creates a natural perimeter following the actual data extent
    edge_blend_distance : int
        Distance in pixels over which to blend terrain to base at NoData edges (default: 10)
        Larger values create smoother edges, smaller values keep sharper terrain boundaries
    
    Returns:
    --------
    dict : Dictionary with solid mesh data ready for 3D printing
        - 'vertices': vertex coordinates
        - 'faces': triangle faces
        - 'top_vertices': count of top surface vertices
        - 'has_base': boolean indicating solid model
        - 'displacement_range': min/max displacement values
    """
    
    # Extract heightmap data from the input dictionary
    heightmap = heightmap_data['heightmap'].astype(float)
    heightmap_original = heightmap.copy()  # Keep original for NoData detection
    h, w = heightmap.shape
    
    # Create NoData mask and smooth it to reduce jagged edges
    nodata_mask = heightmap <= nodata_threshold
    valid_mask = ~nodata_mask
    
    valid_data_count = np.sum(valid_mask)
    total_pixels = heightmap.size
    valid_percent = 100 * valid_data_count / total_pixels
    
    print(f"Heightmap info:")
    print(f"  Dimensions: {w} x {h} pixels")
    print(f"  Valid data: {valid_data_count:,} / {total_pixels:,} pixels ({valid_percent:.1f}%)")
    
    # Smooth the valid mask to reduce spiky edges
    # Apply morphological erosion to shrink the valid region slightly, creating smoother boundaries
    if edge_blend_distance > 0:
        erosion_iterations = max(1, edge_blend_distance // 5)
        smoothed_valid_mask = binary_erosion(valid_mask, iterations=erosion_iterations)
        
        # Create distance field from valid data edges for smooth blending
        # Distance = 0 at NoData, increases into valid region
        distance_from_edge = distance_transform_edt(smoothed_valid_mask).astype(float)
        
        # Create blend factor: 0 at NoData, 1 at valid data, smooth transition
        blend_factor = np.clip(distance_from_edge / edge_blend_distance, 0, 1)
        
        # Apply smoothing to blend factor for even smoother transitions
        blend_factor = gaussian_filter(blend_factor, sigma=2.0)
        
        print(f"  Edge smoothing: {edge_blend_distance}px blend zone, {erosion_iterations} erosion iterations")
    else:
        blend_factor = valid_mask.astype(float)
    
    # Get actual elevation range from the heightmap (only valid data)
    valid_heights = heightmap[valid_mask]
    if len(valid_heights) > 0:
        h_min = np.min(valid_heights)
        h_max = np.max(valid_heights)
        elevation_range = h_max - h_min
    else:
        h_min = 0
        h_max = 1
        elevation_range = 1
    
    print(f"  Elevation range (valid data): {h_min:.2f}m to {h_max:.2f}m (Δ{elevation_range:.2f}m)")
    
    # Calculate appropriate scaling
    if relief_height_mm is not None:
        # User specified desired relief height in mm
        scale_factor = relief_height_mm / elevation_range if elevation_range > 0 else 1.0
        print(f"  Target relief height: {relief_height_mm}mm")
        print(f"  Calculated scale factor: {scale_factor:.6f}")
    else:
        # Fallback
        scale_factor = 0.2
        relief_height_mm = elevation_range * scale_factor
        print(f"  Using default scale factor: {scale_factor}")
        print(f"  Resulting relief height: {relief_height_mm:.2f}mm")
    
    # Normalize heightmap to 0-1 range for valid data
    heightmap_normalized = np.where(
        valid_mask,
        (heightmap - h_min) / (elevation_range if elevation_range > 0 else 1.0),
        0.0
    )
    
    # Create interpolation functions for smooth sampling
    y_coords = np.arange(h)
    x_coords = np.arange(w)
    height_interpolator = RectBivariateSpline(y_coords, x_coords, heightmap_normalized, kx=3, ky=3)
    blend_interpolator = RectBivariateSpline(y_coords, x_coords, blend_factor, kx=3, ky=3)
    
    # Create subdivided grid
    grid_size = grid_subdivisions + 1
    u = np.linspace(0, 1, grid_size)
    v = np.linspace(0, 1, grid_size)
    
    vertices = []
    
    # Generate top surface vertices with displacement and edge blending
    print(f"\nGenerating {grid_size}x{grid_size} displaced grid with smooth edge blending...")
    
    for i, v_coord in enumerate(v):
        for j, u_coord in enumerate(u):
            # Map UV to model space
            x = u_coord * model_size
            y = v_coord * model_size
            
            # Sample heightmap at UV coordinates
            sample_y = v_coord * (h - 1)
            sample_x = u_coord * (w - 1)
            
            # Sample both height and blend factor
            displacement_value = height_interpolator(sample_y, sample_x)[0, 0]
            blend_value = blend_interpolator(sample_y, sample_x)[0, 0]
            
            # Apply blending: NoData regions at z=0, valid terrain at full height
            # blend_value = 0: at base level (z=0, NoData)
            # blend_value = 1: full terrain height (valid data with base_thickness + relief)
            terrain_z = displacement_value * relief_height_mm + base_thickness
            blended_z = terrain_z * blend_value
            
            vertices.append([x, y, blended_z])
    
    # Generate bottom surface vertices (flat base)
    for i, v_coord in enumerate(v):
        for j, u_coord in enumerate(u):
            x = u_coord * model_size
            y = v_coord * model_size
            z = 0.0  # Bottom at z=0
            
            vertices.append([x, y, z])
    
    # Generate faces for top surface
    faces = []
    
    print("Generating top surface faces...")
    for i in range(grid_subdivisions):
        for j in range(grid_subdivisions):
            # Top surface indices
            v0 = i * grid_size + j
            v1 = i * grid_size + (j + 1)
            v2 = (i + 1) * grid_size + j
            v3 = (i + 1) * grid_size + (j + 1)
            
            # Two triangles per quad
            faces.append([v0, v1, v2])
            faces.append([v1, v3, v2])
    
    # Generate faces for bottom surface (inverted normals)
    print("Generating bottom surface faces...")
    bottom_offset = grid_size * grid_size
    
    for i in range(grid_subdivisions):
        for j in range(grid_subdivisions):
            v0 = bottom_offset + i * grid_size + j
            v1 = bottom_offset + i * grid_size + (j + 1)
            v2 = bottom_offset + (i + 1) * grid_size + j
            v3 = bottom_offset + (i + 1) * grid_size + (j + 1)
            
            # Inverted winding order for bottom
            faces.append([v0, v2, v1])
            faces.append([v1, v2, v3])
    
    # Generate side walls to close the mesh (making it watertight/manifold)
    # Use simple rectangular walls for guaranteed watertight geometry
    print("Generating rectangular side walls (watertight)...")
    
    # Front wall (y=0)
    for j in range(grid_subdivisions):
        top_v0 = j
        top_v1 = j + 1
        bottom_v0 = bottom_offset + j
        bottom_v1 = bottom_offset + j + 1
        
        faces.append([top_v0, bottom_v0, top_v1])
        faces.append([top_v1, bottom_v0, bottom_v1])
    
    # Back wall (y=max)
    back_row = grid_subdivisions * grid_size
    for j in range(grid_subdivisions):
        top_v0 = back_row + j
        top_v1 = back_row + j + 1
        bottom_v0 = bottom_offset + back_row + j
        bottom_v1 = bottom_offset + back_row + j + 1
        
        faces.append([top_v0, top_v1, bottom_v0])
        faces.append([top_v1, bottom_v1, bottom_v0])
    
    # Left wall (x=0)
    for i in range(grid_subdivisions):
        top_v0 = i * grid_size
        top_v1 = (i + 1) * grid_size
        bottom_v0 = bottom_offset + i * grid_size
        bottom_v1 = bottom_offset + (i + 1) * grid_size
        
        faces.append([top_v0, top_v1, bottom_v0])
        faces.append([top_v1, bottom_v1, bottom_v0])
    
    # Right wall (x=max)
    right_col = grid_subdivisions
    for i in range(grid_subdivisions):
        top_v0 = i * grid_size + right_col
        top_v1 = (i + 1) * grid_size + right_col
        bottom_v0 = bottom_offset + i * grid_size + right_col
        bottom_v1 = bottom_offset + (i + 1) * grid_size + right_col
        
        faces.append([top_v0, bottom_v0, top_v1])
        faces.append([top_v1, bottom_v0, bottom_v1])
    
    # Calculate displacement statistics
    vertices_array = np.array(vertices)
    actual_min_z = float(np.min(vertices_array[:grid_size*grid_size, 2]))
    actual_max_z = float(np.max(vertices_array[:grid_size*grid_size, 2]))
    actual_relief = actual_max_z - base_thickness
    
    displacement_range = {
        'min_z': actual_min_z,
        'max_z': actual_max_z,
        'relief_height': actual_relief,
        'base_thickness': base_thickness,
        'total_height': actual_max_z
    }
    
    print(f"\nMesh complete: {len(vertices)} vertices, {len(faces)} faces")
    print(f"Model dimensions:")
    print(f"  Base: {model_size}mm x {model_size}mm")
    print(f"  Relief height: {actual_relief:.2f}mm (terrain variation)")
    print(f"  Base thickness: {base_thickness:.2f}mm")
    print(f"  Total height: {actual_max_z:.2f}mm")
    print(f"  Vertical exaggeration: {scale_factor:.2f}x")
    
    return {
        'vertices': vertices,
        'faces': faces,
        'vertex_count': len(vertices),
        'face_count': len(faces),
        'top_vertices': grid_size * grid_size,
        'has_base': True,
        'is_manifold': True,
        'displacement_range': displacement_range,
        'parameters': {
            'grid_subdivisions': grid_subdivisions,
            'relief_height_mm': relief_height_mm,
            'base_thickness': base_thickness,
            'model_size': model_size,
            'vertical_exaggeration': scale_factor,
            'edge_blend_distance': edge_blend_distance
        }
    }


# Example usage - Complete pipeline for 3D printing:
# 
# # Load heightmap from TIFF
# heightmap = import_raster_as_heightmap('data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.tif', scale_z=1.0)
# 
# # Create solid displaced model from heightmap data
# # relief_height_mm=20 means 20mm between lowest and highest point
# # nodata_threshold=0.1 creates clean perimeter (values below 0.1m treated as NoData)
# # edge_blend_distance=10 smooths transitions over 10 pixels
# solid_mesh = create_displaced_grid_model(
#     heightmap,
#     grid_subdivisions=1000,
#     relief_height_mm=20.0,     # 20mm of vertical relief (adjust for more/less drama)
#     base_thickness=5.0,
#     model_size=100.0,
#     nodata_threshold=0.1,       # Values below this are treated as NoData
#     edge_blend_distance=10      # Smooth edge transition over 10 pixels
# )
# 
# # Export to STL
# stl_result = export_mesh_to_stl(solid_mesh, 'terrain_model_3dprint.stl', ascii_format=False)
# print(f"3D printable model exported: {stl_result['file_size_mb']} MB")


In [5]:
def trim_heightmap_edges(heightmap_data, nodata_threshold=0.1):
    """
    Trim NoData/zero-elevation edges from heightmap to remove excess perimeter.
    This creates a cleaner model by removing the flat base areas around actual terrain.
    
    Parameters:
    -----------
    heightmap_data : dict
        Dictionary returned from import_raster_as_heightmap()
    nodata_threshold : float
        Values below this threshold are considered NoData/background (default: 0.1)
    
    Returns:
    --------
    dict : Trimmed heightmap_data with updated dimensions
    """
    
    heightmap = heightmap_data['heightmap']
    h, w = heightmap.shape
    
    # Find rows and columns with valid data
    valid_mask = heightmap > nodata_threshold
    valid_rows = np.any(valid_mask, axis=1)
    valid_cols = np.any(valid_mask, axis=0)
    
    # Find bounding box of valid data
    row_indices = np.where(valid_rows)[0]
    col_indices = np.where(valid_cols)[0]
    
    if len(row_indices) == 0 or len(col_indices) == 0:
        print("Warning: No valid data found. Returning original heightmap.")
        return heightmap_data
    
    row_start, row_end = row_indices[0], row_indices[-1] + 1
    col_start, col_end = col_indices[0], col_indices[-1] + 1
    
    # Crop heightmap to valid region
    trimmed_heightmap = heightmap[row_start:row_end, col_start:col_end]
    
    # Calculate how much was trimmed
    rows_removed = h - (row_end - row_start)
    cols_removed = w - (col_end - col_start)
    percent_removed = 100 * (1 - (trimmed_heightmap.size / heightmap.size))
    
    print(f"Trimming NoData edges:")
    print(f"  Original size: {w} x {h} pixels")
    print(f"  Trimmed size: {col_end-col_start} x {row_end-row_start} pixels")
    print(f"  Removed: {cols_removed} cols, {rows_removed} rows ({percent_removed:.1f}% reduction)")
    
    # Update heightmap_data with trimmed data
    trimmed_data = heightmap_data.copy()
    trimmed_data['heightmap'] = trimmed_heightmap
    trimmed_data['width'] = col_end - col_start
    trimmed_data['height'] = row_end - row_start
    trimmed_data['min_elevation'] = float(np.nanmin(trimmed_heightmap))
    trimmed_data['max_elevation'] = float(np.nanmax(trimmed_heightmap))
    
    return trimmed_data


## Why This Approach is Better for 3D Printing

**Advantages:**
1. **Solid/Manifold Geometry**: Creates watertight mesh with base and walls (required for slicers)
2. **Fine Control**: Adjust `grid_subdivisions` for detail vs. file size balance
3. **UV-Based Displacement**: Smooth interpolation from heightmap texture
4. **Intuitive Relief Control**: `relief_height_mm` directly sets terrain height in mm
5. **Printable Scale**: `model_size` and `base_thickness` ensure practical dimensions
6. **Smooth Edge Transitions**: `edge_blend_distance` creates natural falloff at NoData boundaries
7. **Guaranteed Watertight**: Simple rectangular walls ensure manifold geometry

**Parameter Tuning:**
- `grid_subdivisions=1000`: Good balance (1M vertices). Use 500 for faster testing, 2000+ for maximum detail
- `relief_height_mm=20.0`: Height difference from lowest to highest point in mm. Typical range: 10-50mm for dramatic terrain
- `base_thickness=5.0`: Minimum thickness for stability during printing (5-10mm recommended)
- `model_size=100.0`: Physical size in mm (100mm = 10cm square model)
- `nodata_threshold=0.1`: Heightmap values below this (in meters) are treated as NoData
- `edge_blend_distance=10`: Smooths transitions over N pixels at edges (larger = smoother, reduce spikes)

**Edge Handling:**
- Uses distance transform and Gaussian smoothing for natural terrain falloff
- NoData regions smoothly blend to base thickness (not abrupt cutoff)
- Eliminates spiky edges while preserving terrain shape
- Rectangular walls ensure watertight geometry for all slicers

**vs. Direct Conversion:**
- Direct method creates surface-only mesh (hollow)
- Displacement approach creates solid printable object
- NoData regions properly handled with smooth transitions
- Better for FDM/FFF printing with bases and supports


In [6]:
# Complete workflow example - from preprocessed TIFF to printable STL
from pathlib import Path

# Input file path
input_tiff = 'data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.tif'

# Extract base filename for output
input_name = Path(input_tiff).stem  # Gets filename without extension
output_stl_path = f'data/{input_name}.stl'

print("="*60)
print("Starting STL Generation Pipeline")
print("="*60)
print(f"Input: {input_tiff}")
print(f"Output: {output_stl_path}\n")

# Step 1: Load the heightmap
print("[Step 1/4] Loading heightmap...")
heightmap = import_raster_as_heightmap(input_tiff, scale_z=1.0)
print(f"✓ Loaded: {heightmap['width']}x{heightmap['height']} pixels\n")

# Step 2: Trim excess NoData edges for cleaner model
print("[Step 2/4] Trimming NoData edges...")
heightmap_trimmed = trim_heightmap_edges(heightmap, nodata_threshold=0.1)
print()

# Step 3: Create solid displaced model (watertight, printable)
print("[Step 3/4] Creating 3D solid model...")
solid_model = create_displaced_grid_model(
    heightmap_trimmed,            # Use trimmed heightmap
    grid_subdivisions=1000,       # 1000x1000 grid for good detail
    relief_height_mm=20.0,        # 20mm between lowest and highest point (adjust for more/less drama)
    base_thickness=5.0,           # 5mm solid base
    model_size=100.0,             # 100mm x 100mm footprint
    nodata_threshold=0.1,         # Treat values below 0.1m as NoData
    edge_blend_distance=10        # Smooth edge transitions over 10 pixels (reduce spikes)
)
print()

# Step 4: Export to STL
print("[Step 4/4] Exporting to STL...")
result = export_mesh_to_stl(solid_model, output_stl_path, ascii_format=False)

print("\n" + "="*60)
print("✓ 3D Printable Model Created!")
print("="*60)
print(f"Output file: {result['output_path']}")
print(f"File size: {result['file_size_mb']} MB")
print(f"Geometry: {result['vertex_count']:,} vertices, {result['face_count']:,} faces")
print(f"Relief height: {solid_model['displacement_range']['relief_height']:.2f} mm")
print(f"Total height: {solid_model['displacement_range']['total_height']:.2f} mm")
print(f"Vertical exaggeration: {solid_model['parameters']['vertical_exaggeration']:.2f}x")
print(f"Manifold: {solid_model['is_manifold']}")

# Step 5: Cleanup intermediate files (optional - uncomment to enable)
print("\n[Cleanup] Removing intermediate preprocessing files...")
intermediate_files = [
    'raw_data/mnsterrainbatiment_2015_1m_montreal-est/output_dem_5m.tif',
    'raw_data/mnsterrainbatiment_2015_1m_montreal-est/output_dem_5m_normalized.tif',
    'raw_data/mnsterrainbatiment_2015_1m_montreal-est/output_dem_5m_normalized_reprojected.tif',
    'raw_data/mnsterrainbatiment_2015_1m_montreal-est/output_dem_5m_normalized_reprojected_cleaned.tif',
]

cleaned_count = 0
for f in intermediate_files:
    if os.path.exists(f):
        try:
            os.remove(f)
            cleaned_count += 1
            print(f"  ✓ Deleted: {Path(f).name}")
        except Exception as e:
            print(f"  ✗ Failed to delete {Path(f).name}: {e}")
    
if cleaned_count > 0:
    print(f"\n✓ Cleaned up {cleaned_count} intermediate file(s)")
else:
    print("  No intermediate files found")

print("\n" + "="*60)
print("✓ Pipeline Complete!")
print("="*60)
print(f"\nFinal STL: {output_stl_path}")
print("Ready for slicing in Cura, PrusaSlicer, etc.")


Starting STL Generation Pipeline
Input: data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.tif
Output: data/MNS(terrain+batiment)_2015_1m_Montréal-Est_preprocessed.stl

[Step 1/4] Loading heightmap...
✓ Loaded: 1313x865 pixels

[Step 2/4] Trimming NoData edges...
Trimming NoData edges:
  Original size: 1313 x 865 pixels
  Trimmed size: 1309 x 865 pixels
  Removed: 4 cols, 0 rows (0.3% reduction)

[Step 3/4] Creating 3D solid model...
Heightmap info:
  Dimensions: 1309 x 865 pixels
  Valid data: 558,974 / 1,132,285 pixels (49.4%)
  Edge smoothing: 10px blend zone, 2 erosion iterations
  Elevation range (valid data): 0.12m to 145.34m (Δ145.23m)
  Target relief height: 20.0mm
  Calculated scale factor: 0.137716

Generating 1001x1001 displaced grid with smooth edge blending...
Generating top surface faces...
Generating bottom surface faces...
Generating rectangular side walls (watertight)...

Mesh complete: 2004002 vertices, 4008000 faces
Model dimensions:
  Base: 100.0mm x 100.0